In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

In [4]:
# Question 1: Spark version
spark.version

'3.5.4'

In [5]:
df_yellow = spark.read.parquet("data/raw/")

In [7]:
# Question 2: repartition
df_yellow = df_yellow.repartition(4)
df_yellow.write.parquet("yellow_trips/2024/10/", mode="overwrite")

In [27]:
from pyspark.sql.functions import lit, unix_timestamp, col

In [18]:
df_filtered = df_yellow.filter(col("tpep_pickup_datetime").between("2024-10-15", "2024-10-16"))
df_filtered.count()

128895

In [28]:
#df_yellow_duration = df_yellow.withColumn("duration", datediff(col("tpep_dropoff_datetime"), col("tpep_pickup_datetime")))

df_yellow_duration = df_yellow.withColumn("duration", 
                   (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600)

df_yellow_duration.orderBy(col("duration").desc()).limit(1).show()

[Stage 37:>                                                         (0 + 4) / 4]

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|          duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       2| 2024-10-16 13:03:49|  2024-10-23 07:40:53|              1|        32.37|         3|                 N|          48|    